In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import regex as re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
#import pygwalker as pyg

train = pd.read_csv("combined.csv")
train.head()

,brand,price,old,new,cpu,cpu_brand,ram_capacity,ram_brand,hard_drive_type,hard_drive_capacity,card,card_brand,screen_size,screen_type
0,Dell,15990000.0,0,1,Intel Core i5 1340P,Intel,16GB,LPDDR5,SSD,512 GB,Intel Iris Xe Graphics,Intel,16,FHD
1,Lenovo,27990000.0,0,1,Intel Core i7 1365U,Intel,16GB,LPDDR5,SSD,256 GB,Intel Iris Xe Graphics,Intel,14,FHD
2,Dell,15990000.0,0,1,Intel Core i5 1340P,Intel,16GB,LPDDR5,SSD,512 GB,Intel Iris Xe Graphics,Intel,14,FHD
3,Lenovo,17990000.0,0,1,AMD Ryzen 7 7735H,AMD,16GB,LPDDR5,SSD,512 GB,AMD Radeon 680M,AMD,14,3K
4,Microsoft,26990000.0,0,1,Intel Core i7 1255U,Intel,16GB,LPDDR5,SSD,256 GB,Intel Iris Xe Graphics,Intel,13,3K


In [123]:
train['brand'] = train['brand'].replace('Apple', 'Macbook')
train['brand'] = train['brand'].replace('MacBook', 'Macbook')
train['brand'] = train['brand'].replace('Thinkpad', 'Lenovo')
train['brand'] = train['brand'].replace('Legion', 'Lenovo')
train['brand'] = train['brand'].replace('Ideapad', 'Lenovo')
train['brand'] = train['brand'].replace('ThinkPad', 'Lenovo')
train['brand'] = train['brand'].replace('LENOVO', 'Lenovo')
train['brand'] = train['brand'].replace('ACER', 'Acer')
train['brand'] = train['brand'].replace('ASUS', 'Asus')
train['price'] = np.log(train['price'])


In [124]:
unique_brands = train['brand'].unique()
print(unique_brands)

['Dell' 'Lenovo' 'Microsoft' 'Macbook' 'Acer' 'LG' 'Asus' 'MSI' 'HP'
 'Gaming' nan]


In [125]:
brand_counts = train['brand'].value_counts()
print(brand_counts)

brand
Lenovo       328
Asus         170
Dell         140
HP            84
Acer          79
MSI           72
LG            19
Gaming        18
Macbook       17
Microsoft     10
Name: count, dtype: int64


In [126]:
train.columns

Index(['brand', 'price', 'old', 'new', 'cpu', 'cpu_brand', 'ram_capacity',
       'ram_brand', 'hard_drive_type', 'hard_drive_capacity', 'card',
       'card_brand', 'screen_size', 'screen_type'],
      dtype='object')

In [127]:
cat_cols = ['brand', 'cpu', 'cpu_brand', 'ram_capacity',
       'ram_brand', 'hard_drive_type', 'hard_drive_capacity', 'card',
       'card_brand', 'screen_size', 'screen_type']

In [128]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                937 non-null    object 
 1   price                920 non-null    float64
 2   old                  940 non-null    int64  
 3   new                  940 non-null    int64  
 4   cpu                  937 non-null    object 
 5   cpu_brand            937 non-null    object 
 6   ram_capacity         937 non-null    object 
 7   ram_brand            900 non-null    object 
 8   hard_drive_type      936 non-null    object 
 9   hard_drive_capacity  887 non-null    object 
 10  card                 927 non-null    object 
 11  card_brand           927 non-null    object 
 12  screen_size          934 non-null    object 
 13  screen_type          863 non-null    object 
dtypes: float64(1), int64(2), object(11)
memory usage: 102.9+ KB


In [129]:
print('trainframe encoded by OHE dimension : ', pd.get_dummies(train, columns = cat_cols, drop_first = True).shape)

trainframe encoded by OHE dimension :  (940, 758)


In [130]:
train[cat_cols] = train[cat_cols].fillna('missing')

In [131]:
en = LabelEncoder()

for cols in cat_cols:
    train[cols] = en.fit_transform(train[cols])

print('Dataframe encoded by Label encoding dimension : ', train.shape)

Dataframe encoded by Label encoding dimension :  (940, 14)


In [142]:
train.head()

,brand,price,old,new,cpu,cpu_brand,ram_capacity,ram_brand,hard_drive_type,hard_drive_capacity,card,card_brand,screen_size,screen_type
0,2,16.587474,0,1,232,5,4,21,6,14,96,14,20,8
1,6,17.147358,0,1,266,5,4,21,6,9,96,14,9,8
2,2,16.587474,0,1,232,5,4,21,6,14,96,14,9,8
3,6,16.705327,0,1,19,0,4,21,6,14,8,5,9,6
4,9,17.110977,0,1,254,5,4,21,6,9,96,14,3,6


In [133]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                940 non-null    int64  
 1   price                920 non-null    float64
 2   old                  940 non-null    int64  
 3   new                  940 non-null    int64  
 4   cpu                  940 non-null    int64  
 5   cpu_brand            940 non-null    int64  
 6   ram_capacity         940 non-null    int64  
 7   ram_brand            940 non-null    int64  
 8   hard_drive_type      940 non-null    int64  
 9   hard_drive_capacity  940 non-null    int64  
 10  card                 940 non-null    int64  
 11  card_brand           940 non-null    int64  
 12  screen_size          940 non-null    int64  
 13  screen_type          940 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 102.9 KB


In [134]:
print("Max value in y_train:", train.max())
print("Min value in y_train:", train.min())


Max value in y_train: brand                   10.00000
price                   18.81939
old                      1.00000
new                      1.00000
cpu                    338.00000
cpu_brand               16.00000
ram_capacity            16.00000
ram_brand               28.00000
hard_drive_type          7.00000
hard_drive_capacity     23.00000
card                   248.00000
card_brand              28.00000
screen_size             43.00000
screen_type              9.00000
dtype: float64
Min value in y_train: brand                   0.000000
price                  15.759991
old                     0.000000
new                     0.000000
cpu                     0.000000
cpu_brand               0.000000
ram_capacity            0.000000
ram_brand               0.000000
hard_drive_type         0.000000
hard_drive_capacity     0.000000
card                    0.000000
card_brand              0.000000
screen_size             0.000000
screen_type             0.000000
dtype: float64


In [135]:
X = train.drop('price', axis = 1)
y = train['price']

In [136]:
print(X.shape)
print(y.shape)

(940, 13)
(940,)


In [137]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.05,random_state=0)

In [138]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(893, 13)
(893,)
(47, 13)
(47,)


In [139]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

In [140]:
lgbmr = LGBMRegressor()
xgbr = XGBRegressor()

In [141]:
lgbmr.fit(X_train,y_train)
xgbr.fit(X_train,y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000394 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 496
[LightGBM] [Info] Number of data points in the train set: 893, number of used features: 13
[LightGBM] [Info] Start training from score 16.637605


XGBoostError: [23:55:19] /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/data/data.cc:514: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000030110f1f0 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x0000000301275adc xgboost::MetaInfo::SetInfoFromHost(xgboost::Context const&, xgboost::StringView, xgboost::Json) + 3644
  [bt] (2) 3   libxgboost.dylib                    0x0000000301274acc xgboost::MetaInfo::SetInfo(xgboost::Context const&, xgboost::StringView, xgboost::StringView) + 472
  [bt] (3) 4   libxgboost.dylib                    0x0000000301126fb0 XGDMatrixSetInfoFromInterface + 240
  [bt] (4) 5   libffi.8.dylib                      0x000000010590404c ffi_call_SYSV + 76
  [bt] (5) 6   libffi.8.dylib                      0x0000000105901834 ffi_call_int + 1404
  [bt] (6) 7   _cty